# Conditionnal Bernoulli

## question 1

`définition des objet`  
Let $N$ be a natural number, and consider the probability vector $p = (p_1, p_2, \dots, p_N)$ where each $p_i$ belongs to the interval $(0,1)$ for  $i = 1, \dots, N$.  
The sample space is given by $\Omega = \{0,1\}^N.$  
We assume that $I$ is an integer satisfying $I \leq \frac{N}{2}$.  
Let $g$ be the probability density function of the random vector  
$$
(X_1, \dots, X_N)
$$
where the $X_i$ are independent Bernoulli random variables $X_i \sim \mathsf{B}(p_i).$  
Finally, let $f$ be the probability density function of  
$$
(X_1, \dots, X_N) \sim \mathsf{CB}(p, I).
$$  

`justification du choix de ce que l'on fait`
We will use the following proposition from the lecture :
> Let $f$, $g$ be probability density functions (PDFs) such that  
> 
> $$
> f \leq M g \quad \text{with } M \geq 1.
> $$
> 
> ### Accept-Reject Algorithm
> Repeat:  
> 1. Draw $X \sim g$ and $U \sim U[0,1]$.  
> 2. Until $U \leq \frac{f(X)}{M g(X)}$.
> 
> ### Properties:
> - $X \sim f$ 
> - The number of draws until acceptance follows a **Geometric** distribution:  
>   
>   $$
>   \text{Geometric}(1/M).
>   $$  

This propriety can be used because $g$ does not cancel out `j'imagine que c'est comme ça que se dit ne s'annule pas` on $\Omega$.  
We choose the smallest possible $M$ in order to have as little draws as possible 
$$
M = \underset{x \in \Omega}{\sup} \; \frac{f(x)}{g(x)}
$$

With such a $M$, in the algorithm, $X$ is rejected if and only if $f(X)$ is null. In other words, $X$ is rejected if and only if there is not exactly $I$ 1 in it. As such the algorithm can be simplified as:

> ### Accept-Reject Algorithm
> Repeat:  
> 1. Draw $X \sim g$
> 2. Until $|| X|| _1 = I$.


### algorithm du cours aucun changement

In [1]:
import numpy as np
import itertools

In [2]:
def generate_sequences(N, I):
    # Generate all combinations of I positions out of N
    # This will give the indices of the positions that should be 1
    positions = itertools.combinations(range(N), I)
    
    # Generate the sequences by setting the corresponding positions to 1
    sequences = []
    for pos in positions:
        seq = np.zeros(N, dtype=int)  # Start with a list of all zeros
        seq[list(pos)] = 1  # Set the specified positions to 1
        sequences.append(seq)
    
    return np.array(sequences)


def pdf_bernoulli(p):
    def g(x):
        return np.prod(np.where(x==1,p,1-p))
    return g

def pdf_cb(p,I):
    g=pdf_bernoulli(p)
    proba = 0
    for x in generate_sequences(len(p),I):
        proba=proba+g(x)
    def f(x):
        if sum(x) !=I:
            return 0
        else:
            return g(x)/proba
    return f

In [3]:
def rejection_sampling(p,I,M,L):
    samples =[]
    g = pdf_bernoulli(p)
    f = pdf_cb(p,I)
    
    while len(samples) < L:
        X = np.array([np.random.binomial(1, p_i) for p_i in p])
        U = np.random.uniform(0,1)
        if U<=(f(X)/(g(X)*M)):
            samples.append(X)
    return np.array(samples)

In [4]:
N = 3
I=2
L=100000
p = np.random.uniform(0,1,N)

g = pdf_bernoulli(p)
f = pdf_cb(p,I)
M= 100
for seq in generate_sequences(N,I):
    pr=f(seq)/g(seq)
    if pr<=M:
        M=pr

print(p)
print(M)
samples = rejection_sampling(p,I,M,L)
np.mean(samples, axis = 0)


[0.76084536 0.96560862 0.21866482]
1.5866761258723607


array([0.92173, 0.99066, 0.08761])

### algorithm simplifié

In [5]:
#méthode simple du on prends N binomial indépendante puis on vérifie si leurs somme est égal à I, si oui c'est bon, si non on recommence
def rejection_sampling(p,I,L):
    samples =[]
    while len(samples) < L:
        X = np.array([np.random.binomial(1, p_i) for p_i in p])
        if X.sum() == I:
            samples.append(X)
    return np.array(samples)

In [6]:
# évaluation de si cette méthode donne bien un truc binomial avec les bonne probas
N = 3
I=2
L=100000
p = np.random.uniform(0,1,N)
print(p)
samples = rejection_sampling(p,I,L)
np.mean(samples, axis = 0)

[0.84400312 0.297513   0.13567604]


array([0.97992, 0.73573, 0.28435])

Ca ne marche vrmt pas, mais en fait les deux méthode revienne à la même chose car pour le M bien choisi on a U>f(x)/g(x)*M ssi f(x)=0 et donc il suffit de faire tourner des bernoulli jusqu'à qu'on ait un suite de bernoulli qui ait le bon nombre de 1

### conclusion

Ces deux algo (dont les résultats devraient, si j'ai pas fait d'erreur dans mon raisonnement, être exactement les même), il va falloir étudier les résultats prochainement.